In [1]:
import graphlab
import numpy as np
import pandas as pd
import matplotlib 

In [2]:
# Drop the duplicates 
Total_Test = pd.read_csv("../Total_Test.csv")
Total_Test = Total_Test.drop_duplicates()
Total_Test['question_id'] = Total_Test['question_id'].astype(str)
Total_Test.to_csv('../Total_Test.csv', encoding='utf-8', index = False)

/Users/gaozhipeng/anaconda/envs/dato-env/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (1,2,3,4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
Total_Test = pd.read_csv("../Total_Test.csv")

In [5]:
def Format_Voted():
    answer_voted = Total_Test[['question_id','question_content','question_tags','answer_voted' ]]
    answer_voted.to_csv('./Answer_Voted.csv', encoding='utf-8', index = False)
    
    voted = graphlab.SFrame.read_csv('./Answer_Voted.csv')
    been_voted =  voted[ voted['answer_voted'] > 0 ] 
    un_voted = voted[ voted['answer_voted'] == 0 ]
    
    # been_voted = been_voted.remove_column('answer_voted') # 18134
    # un_voted = un_voted.remove_column('answer_voted')  # 18123
    
    # No drop duplicates save as 'been_voted.csv' & 'un_voted.csv' 
    been_voted.save('./been_voted.csv', format='csv')
    un_voted.save('./un_voted.csv', format='csv')
    
    # Dorp duplicates save as 'Been_Voted.csv' & 'Un_Voted.csv'
    Been_Voted = pd.read_csv('./been_voted.csv')
    Un_Voted = pd.read_csv('./un_voted.csv')
    
    Been_Voted = Been_Voted.drop('answer_voted', 1)
    Un_Voted = Un_Voted.drop('answer_voted', 1)
    Been_Voted = Been_Voted.drop_duplicates()
    Un_Voted = Un_Voted.drop_duplicates()
    Been_Voted.to_csv('./Been_voted.csv', encoding='utf-8', index = False )
    Un_Voted.to_csv('./Un_voted.csv', encoding='utf-8', index = False )

In [6]:
Format_Voted()

2016-04-13 21:43:14,385 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460554990.log


Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/SIMILAR/Answer_Voted.csv

Parsing completed. Parsed 100 lines in 0.184462 secs.

This non-commercial license of GraphLab Create is assigned to beyond_acm@163.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Unable to parse line "question_id,question_content,question_tags,answer_voted"

1 lines failed to parse correctly

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/SIMILAR/Answer_Voted.csv

Parsing completed. Parsed 39568 lines in 0.216948 secs.


Inferred types from first line of file as 
column_type_hints=[int,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [7]:
# Total_Test

In [8]:
question_content = Total_Test[['question_id','question_content','question_tags']]
question_content = question_content.drop_duplicates()
question_content.to_csv('../Question_Content.csv', encoding='utf-8', index = False)

In [9]:
question_content = graphlab.SFrame.read_csv('../Question_Content.csv')

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/Question_Content.csv

Parsing completed. Parsed 100 lines in 0.147483 secs.

Unable to parse line "question_id,question_content,question_tags"

1 lines failed to parse correctly

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/Question_Content.csv

Parsing completed. Parsed 31012 lines in 0.177346 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [10]:
# question_content   # 31012 x 3

In [11]:
# add 'text' && 'word_count' && 'tfidf' features
question_content['text'] = question_content['question_tags'] + question_content['question_content']
question_content['word_count'] = graphlab.text_analytics.count_words(question_content['text'])

In [12]:
tfidf = graphlab.text_analytics.tf_idf(question_content['word_count'])

In [13]:
question_content['tfidf'] = tfidf

In [16]:
# question_content  # 31012 x 6

In [17]:
been_voted = graphlab.SFrame.read_csv('./Been_voted.csv')
un_voted = graphlab.SFrame.read_csv('./Un_voted.csv')

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/SIMILAR/Been_voted.csv

Parsing completed. Parsed 100 lines in 0.090971 secs.

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/SIMILAR/Been_voted.csv

Parsing completed. Parsed 16358 lines in 0.096213 secs.

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/SIMILAR/Un_voted.csv

Parsing completed. Parsed 100 lines in 0.080545 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /Users/gaozhipeng/ML/SVM_RANK/SIMILAR/Un_voted.csv

Parsing completed. Parsed 18063 lines in 0.105444 secs.


Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [18]:
# len(been_voted)
# been_voted
# un_voted   # 18063 x 3
# been_voted   # 16358 x 3

In [19]:
def Select_Question( num_question ):
    selected_question = question_content[ question_content['question_id'] == been_voted[0:1]['question_id'][0]]
    for i in range(num_question) :
        selected_qid = been_voted[i*300 + 88]['question_id']
        selected_content = question_content[ question_content['question_id'] == selected_qid ]
        selected_question = selected_question.append(selected_content)
    return selected_question

In [20]:
# Random Select 50 voted / Unvoted Question
selected = Select_Question(49)

In [22]:
# len( selected )
# selected  # 50 x 6

In [23]:
knn_model = graphlab.nearest_neighbors.create(question_content, features=['tfidf'], label='question_id')

Starting brute force nearest neighbors model training.

In [24]:
Similar_Qid_List = {}
for i in range( len(selected) ) :
    key = 'Q' + str(i) + '_Similar_List'
    Q_i = selected[i:i+1]
    value = knn_model.query(Q_i, k=20)['reference_label']
    Similar_Qid_List[key] = value

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00322456  | 24.893ms     |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00322456  | 16.038ms     |

| Done         |         | 100         | 73.539ms     |

+--------------+---------+-------------+--------------+

In [25]:
def Get_Index_QuestionID(qid) :
    return Total_Test[ Total_Test['question_id'] == qid].index.values

In [26]:
def Get_IndexList(Q_List) :
    IndexList = []
    for qid in Q_List:
        indexes = Get_Index_QuestionID( str(qid) )
        if len(indexes) == 0:
            indexes = Get_Index_QuestionID( qid )
        # print qid, indexes
        for index in indexes:
            IndexList.append(index)
            # print len( IndexList )
    return IndexList

In [29]:
# Similar_Qid_List

In [30]:
# From Similar Qid List Found Similar Index 
Similar_Index_List = {}
for i in range( len(Similar_Qid_List) ) :
    key = 'Q' + str(i) + '_Similar_List'
    # value is a list of similar qid
    Qid_List = Similar_Qid_List[key]
    Index_List = Get_IndexList(Qid_List)
    Similar_Index_List[key]  = Index_List    

In [32]:
# len( Similar_Index_List )
# Similar_Index_List

In [33]:
# import json
# import pickle

def writeDict(dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( dict.items() ):
            f.write( str(key) + sep + ','.join(str(x) for x in value) + '\n' )

In [34]:
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return(dict)

In [35]:
writeDict(Similar_Qid_List, './Similar_Qid_List.txt', ':')
writeDict(Similar_Index_List, './Similar_Index_List.txt', ':')

In [36]:
test = readDict('./Similar_Qid_List.txt', ':')

In [37]:
test == Similar_Qid_List

True